### 1. 조회수 100위까지 자르고, 태그 수를 비교/어떤 태그가 많이 사용되었는지 알아보기

1) 파일 불러오기

In [2]:
import pandas as pd
input_fd = open('KRvideos.csv',errors = 'backslashreplace',encoding='utf-8')
kr = pd.read_csv(input_fd)

In [16]:
print(kr.shape)

(34567, 16)


2) 조회수 정렬 후 100위까지 자르기

In [56]:
kr100 = kr.sort_values('views', ascending=False).iloc[:100,:]

In [54]:
kr100.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
4856,FlsCjmMhFmw,17.11.12,YouTube Rewind: The Shape of 2017 | #YouTubeRe...,YouTube Spotlight,24,2017-12-06T17:58:51.000Z,"Rewind|""Rewind 2017""|""youtube rewind 2017""|""#Y...",113876217,2811223,1470388,787174,https://i.ytimg.com/vi/FlsCjmMhFmw/default.jpg,False,False,False,"YouTube Rewind 2017. Celebrating the videos, p..."
4678,FlsCjmMhFmw,17.10.12,YouTube Rewind: The Shape of 2017 | #YouTubeRe...,YouTube Spotlight,24,2017-12-06T17:58:51.000Z,"Rewind|""Rewind 2017""|""youtube rewind 2017""|""#Y...",100911567,2656686,1353663,682890,https://i.ytimg.com/vi/FlsCjmMhFmw/default.jpg,False,False,False,"YouTube Rewind 2017. Celebrating the videos, p..."
4018,6ZfuNTqbHE8,17.06.12,Marvel Studios' Avengers: Infinity War Officia...,Marvel Entertainment,24,2017-11-29T13:26:24.000Z,"marvel|""comics""|""comic books""|""nerdy""|""geeky""|...",87450245,2584675,52176,341571,https://i.ytimg.com/vi/6ZfuNTqbHE8/default.jpg,False,False,False,There was an idea… Avengers: Infinity War. In ...
30844,7C2z4GqqS5E,18.24.05,BTS (방탄소년단) 'FAKE LOVE' Official MV,ibighit,10,2018-05-18T09:00:02.000Z,"BIGHIT|""빅히트""|""방탄소년단""|""BTS""|""BANGTAN""|""방탄""|""FAK...",87430865,5150839,172570,1142274,https://i.ytimg.com/vi/7C2z4GqqS5E/default.jpg,False,False,False,BTS (방탄소년단) 'FAKE LOVE' Official MVDirector : ...
3750,6ZfuNTqbHE8,17.05.12,Marvel Studios' Avengers: Infinity War Officia...,Marvel Entertainment,24,2017-11-29T13:26:24.000Z,"marvel|""comics""|""comic books""|""nerdy""|""geeky""|...",84281319,2555414,51008,339708,https://i.ytimg.com/vi/6ZfuNTqbHE8/default.jpg,False,False,False,There was an idea… Avengers: Infinity War. In ...


In [57]:
kr100.shape

(100, 16)

3) 태그 수를 집계한 tag_count 컬럼 추가하기

In [58]:
kr100['tag_count'] = kr100['tags'].apply(lambda x:x.count('|')+1)

In [59]:
print(kr100.iloc[1].tags)
print(kr100.iloc[1].tag_count)

Rewind|"Rewind 2017"|"youtube rewind 2017"|"#YouTubeRewind"|"Rewind 2016"|"Dan and Phil"|"Grace Helbig"|"HolaSoyGerman"|"Lilly Singh"|"Markiplier"|"Swoozie"|"Rhett Link"|"Liza Koshy"|"Dolan Twins"|"Lele Pons"|"Jake Paul"|"Logan Paul"|"KSI"|"Joey Graceffa"|"Casey Neistat"|"Poppy"|"Niana Guerrero"|"Daddy Yankee"|"Luis Fonsi"|"Ed Sheeran"|"Kendrick Lamar"|"Stephen Colbert"|"Fidget Spinners"|"Slime"|"Backpack Kid"|"April the Giraffe"|"#Rewind"|"Despacito"|"Shape of you"|"YouTubeRewind"|"I’m the One"|"Humble"
37


#### 1-1. 태그 수와 조회수의 상관관계 분석하기 

In [60]:
kr100['ranking_group']=sorted(list(range(0,10))*10)

In [64]:
print(kr100.groupby('ranking_group').tag_count.mean())

ranking_group
0    18.1
1     8.1
2    23.0
3    17.3
4    14.1
5    13.2
6    20.0
7    19.9
8    15.4
9    17.3
Name: tag_count, dtype: float64


조회수 상위 100개 동영상에서 10개씩 동영상을 그룹화하여 태그 수를 비교하였을 때, 태그 수에 따른 조회수 차이는 없는 것으로 보인다.

#### 1-2. 조회수 상위 100개 동영상에서 많이 사용된 태그 찾기

In [80]:
def string_handle(x): #문자열 처리 함수
    x = x.lower() #소문자로 변경
    x = x.replace('"','') #"제거
    x = x.replace('#','') ##제거
    x = x.replace('_', ' ') #_를 공백으로 변경
    x = x.strip() #양쪽 공백 제거
    return x

In [74]:
tag100 = []
for tags in kr100['tags']:
    tag_lst = tags.split('|')
    tag100.extend(tag_lst)

In [81]:
handled_tag100 = [string_handle(x) for x in tag100]

In [119]:
tag100_count=pd.DataFrame(data={'tag':handled_tag100}).groupby('tag').size().reset_index().rename(columns={0:'count'}).sort_values('count',ascending=False)
tag100_count.head(15)

,tag,count
250,marvel,26
360,superhero,18
149,fake love,18
359,super hero,18
72,comics,18
71,comic books,18
202,infinity war,18
26,bangtan,15
32,bighit,15
44,bts,15


많이 사용된 태그 상위 15개에서 pop을 제외한 14개 모두 marvel이나 방탄소년단과 연관된 태그이다. 17~18년도 한국 유튜브 트랜드는 마블 영화인 어밴져스와 방탄소년단이 지배적이였던 것으로 보인다.

#### 1-3. 캐나다, 미국, 영국의 조회수 상위 100개 영상의 태그 트렌드 분석

In [122]:
ca=pd.read_csv('CAvideos.csv')
us=pd.read_csv('USvideos.csv')
gb=pd.read_csv('GBvideos.csv')

In [128]:
def top_tag10(data):
    df100 = data.sort_values('views', ascending=False).iloc[:100,:]
    tag100 = []
    for tags in df100['tags']:
        tag_lst = tags.split('|')
        tag100.extend(tag_lst)
    handled_tag100 = [string_handle(x) for x in tag100]
    tag100_count=pd.DataFrame(data={'tag':handled_tag100}).groupby('tag').size().reset_index().rename(columns={0:'count'}).sort_values('count',ascending=False)
    return tag100_count.iloc[:10,:]

In [136]:
top_tag10(ca)

,tag,count
306,pop,21
264,marvel,19
387,superhero,14
386,super hero,14
85,comics,14
84,comic books,14
189,infinity war,14
27,avengers,14
126,ed sheeran,13
464,youtuberewind,12


캐나다도 한국과 마찬가지로 marvle의 어밴저스가 지배적인것을 볼 수 있다. 더불어 Ed Sheeran의 노래도 인기를 끈 것으로 보인다.

In [137]:
top_tag10(us)

,tag,count
57,fake love,34
137,rap,29
112,mcdj recording/rca records,22
159,this is america,22
35,childish gambino,22
135,pop,21
138,records,16
15,bts,12
38,cry,12
131,no,12


미국은 오히려 순위권에 marvel 관련 태그가 없고 비교적 태그들이 다양하게 분포하고 있다. 다른 나라의 유튜브 트렌드와 다른 점은 순위권에 있는 많은 태그가 대중음악와 관련되어 있다는 점이다. 방탄소년단의 fake love, Childish Gambino의 This is America가 인기를 끌었고, 그 밖에도 rap, pop, recoreds 등 대부분의 태그가 음악과 관련되어 있다.

In [138]:
top_tag10(gb)

,tag,count
4,bad bunny,54
60,ozuna,45
1,amorfoda,42
69,romeo santos,33
26,el farsante remix,33
62,piso 21,22
68,reik,22
23,dura,22
24,déjala que vuelva,22
25,ed sheeran,22


영국도 미국과 마찬가지로 상위 100개 영상의 대부분의 태그가 대중음악과 관련된 태그였다. 영국에는 가수 bad bunny의 amorfoda라는 노래가 선풍적인 인기를 끈 것으로 보인다. 100개 영상 중 거의 절반이 이 곡과 관련된 태그를 가지고 있었다. 영국 유튜브에서 의외인 점은 태그 대부분이 스페인 노래와 관련되었다는 점이다.